In [ ]:
import pandas as pd

df = pd.read_csv('data/messages.csv')

## Goal

Goal is to split the timestamp column. I want the following in separate columns:
- Date
- Time
- Read By Time

In [ ]:
import re
test = "Jun 26, 2024  1:15:45 PM (Read by you after 3 minutes, 18 seconds)"

def SeparateTimestamp(timestamp):
    date = re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\s]\d{2,4},\s\d{4}', timestamp)
    time = re.search(r'\d{1,2}:\d{2}:\d{2} (?:AM|PM)', timestamp)
    readTimeTemp = re.search(r'\([^)]*\)', timestamp)
    
    readTime = None
    if readTimeTemp is not None:
        readTime = readTimeTemp.group(0)
    
    return date.group(0), time.group(0), readTime

In [ ]:
df['timestamp'] = df['timestamp'].apply(lambda x: SeparateTimestamp(x))

In [ ]:
df[['date', 'time', 'read_time']] = df['timestamp'].apply(pd.Series)

In [ ]:
df.drop(columns=['timestamp'], inplace=True)

### Added read_time_in_seconds for easier analysis

In [ ]:
def ExtractTime(string):
    time_patterns = {
        'hour': re.compile(r'(\d+)\s*hour'),
        'minute': re.compile(r'(\d+)\s*minute'),
        'second': re.compile(r'(\d+)\s*second')
    }
    
    time_dict = {}
    for unit, pattern in time_patterns.items():
        match = pattern.search(string)
        if match:
            time_dict[unit] = match.group(1)
    return time_to_seconds(time_dict)
    
def time_to_seconds(time_dict):
    seconds = 0
    if 'hour' in time_dict:
        seconds += int(time_dict['hour']) * 3600
    if 'minute' in time_dict:
        seconds += int(time_dict['minute']) * 60
    if 'second' in time_dict:
        seconds += int(time_dict['second'])
    return seconds


In [ ]:
df['read_time_in_seconds'] = df['read_time'].apply(lambda x: ExtractTime(x) if (x != None) else None)

In [ ]:
df['sender'] = df['sender'].apply(lambda x: 'Me' if x == 'Me' else 'My Wife')
df

In [ ]:
df.to_csv('data/cleaned_messages.csv', index=False)